In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# list of files to combine
Citibike24 = ['datalast/JC-202404-citibike-tripdata.csv','datalast/JC-202304-citibike-tripdata.csv']

In [3]:
# create empty list
dfCitibike24_list = []

In [4]:
# for loop to read the list of files int dataframe
for file in Citibike24:
    dfcitibike24 = pd.read_csv(file)
    dfCitibike24_list.append(dfcitibike24)

In [5]:
# concatenate all 6 months dataframe
dfcitibike24 = pd.concat(dfCitibike24_list,axis=0)

In [6]:
# show dataframe
dfcitibike24.dropna(inplace=True)
dfcitibike24.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,2D398FDF45B98978,electric_bike,2024-04-16 21:43:36,2024-04-16 22:03:44,Marin Light Rail,JC013,Hoboken Terminal - River St & Hudson Pl,HB102,40.714584,-74.042817,40.736068,-74.029127,casual
1,7D463681098397A7,classic_bike,2024-04-20 20:06:34,2024-04-20 20:20:33,Heights Elevator,JC059,Bergen Ave & Sip Ave,JC109,40.748716,-74.040443,40.731009,-74.064437,member
2,5EED2D897C650AEE,classic_bike,2024-04-09 07:56:59,2024-04-09 07:59:43,6 St & Grand St,HB302,Clinton St & Newark St,HB409,40.744398,-74.034501,40.737430,-74.035710,member
3,E1D43CEADADF8D3C,electric_bike,2024-04-17 08:44:36,2024-04-17 08:51:06,6 St & Grand St,HB302,Newport PATH,JC066,40.744318,-74.034337,40.727224,-74.033759,member
4,B49012F99F97FEE4,classic_bike,2024-04-13 08:50:30,2024-04-13 08:55:10,Lafayette Park,JC078,Liberty Light Rail,JC052,40.713464,-74.062859,40.711242,-74.055701,member


In [7]:
# show dataframe info
dfcitibike24.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159365 entries, 0 to 80687
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             159365 non-null  object 
 1   rideable_type       159365 non-null  object 
 2   started_at          159365 non-null  object 
 3   ended_at            159365 non-null  object 
 4   start_station_name  159365 non-null  object 
 5   start_station_id    159365 non-null  object 
 6   end_station_name    159365 non-null  object 
 7   end_station_id      159365 non-null  object 
 8   start_lat           159365 non-null  float64
 9   start_lng           159365 non-null  float64
 10  end_lat             159365 non-null  float64
 11  end_lng             159365 non-null  float64
 12  member_casual       159365 non-null  object 
dtypes: float64(4), object(9)
memory usage: 17.0+ MB


In [8]:
# Update new header member_casual to align with the older name, usertype
dfcitibike24.rename(columns={"member_casual":"user_type"}, inplace=True)
dfcitibike24.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,user_type
0,2D398FDF45B98978,electric_bike,2024-04-16 21:43:36,2024-04-16 22:03:44,Marin Light Rail,JC013,Hoboken Terminal - River St & Hudson Pl,HB102,40.714584,-74.042817,40.736068,-74.029127,casual
1,7D463681098397A7,classic_bike,2024-04-20 20:06:34,2024-04-20 20:20:33,Heights Elevator,JC059,Bergen Ave & Sip Ave,JC109,40.748716,-74.040443,40.731009,-74.064437,member
2,5EED2D897C650AEE,classic_bike,2024-04-09 07:56:59,2024-04-09 07:59:43,6 St & Grand St,HB302,Clinton St & Newark St,HB409,40.744398,-74.034501,40.737430,-74.035710,member
3,E1D43CEADADF8D3C,electric_bike,2024-04-17 08:44:36,2024-04-17 08:51:06,6 St & Grand St,HB302,Newport PATH,JC066,40.744318,-74.034337,40.727224,-74.033759,member
4,B49012F99F97FEE4,classic_bike,2024-04-13 08:50:30,2024-04-13 08:55:10,Lafayette Park,JC078,Liberty Light Rail,JC052,40.713464,-74.062859,40.711242,-74.055701,member


In [9]:
# adding empty column Duration
dfcitibike24["trip_duration"] = ""

In [10]:
dfcitibike24["trip_duration"] = (pd.to_datetime(dfcitibike24["ended_at"]) - pd.to_datetime(dfcitibike24["started_at"])).dt.total_seconds()/60

In [11]:
# adding empty column Distance
dfcitibike24["distance_in_miles"] = ""

In [12]:
# Calculating & appending Distance
# Approximate radius of earth in km
R = 6373.0

lat1 = np.radians(dfcitibike24["start_lat"])
lat2 = np.radians(dfcitibike24["end_lat"])
lng1 = np.radians(dfcitibike24["start_lng"])
lng2 = np.radians(dfcitibike24["end_lng"])
dlng = lng2 - lng1
dlat = lat2 - lat1

a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

dfcitibike24["distance_in_miles"] = R*c

R=6373
for index,row in dfcitibike23.iterrows():
    lat1=radians(row["start_lat"])
    lon1=radians(row["start_lng"])
    lat2=radians(row["end_lat"])
    lon2=radians(row["end_lng"])
    
    dlon=lon2-lon1
    dlat=lat2-lat1
    
    a=sin(dlat/2)**2 + cos(lat2)*sin(dlon/2)**2
    c=2*atan2(sqrt(a),sqrt(1-a))
    
    dfcitibike23.at[index,"distance"]=R*c

In [13]:
# Format the trip_duration and distance_in_miles to 2 decimal places
dfcitibike24['trip_duration'] = dfcitibike24['trip_duration'].apply(lambda x: format(x, '.2f'))
dfcitibike24['distance_in_miles'] = dfcitibike24['distance_in_miles'].apply(lambda x: format(x, '.2f'))

In [14]:
# show dataframe
dfcitibike24.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,user_type,trip_duration,distance_in_miles
0,2D398FDF45B98978,electric_bike,2024-04-16 21:43:36,2024-04-16 22:03:44,Marin Light Rail,JC013,Hoboken Terminal - River St & Hudson Pl,HB102,40.714584,-74.042817,40.736068,-74.029127,casual,20.13,2.65
1,7D463681098397A7,classic_bike,2024-04-20 20:06:34,2024-04-20 20:20:33,Heights Elevator,JC059,Bergen Ave & Sip Ave,JC109,40.748716,-74.040443,40.731009,-74.064437,member,13.98,2.82
2,5EED2D897C650AEE,classic_bike,2024-04-09 07:56:59,2024-04-09 07:59:43,6 St & Grand St,HB302,Clinton St & Newark St,HB409,40.744398,-74.034501,40.737430,-74.035710,member,2.73,0.78
3,E1D43CEADADF8D3C,electric_bike,2024-04-17 08:44:36,2024-04-17 08:51:06,6 St & Grand St,HB302,Newport PATH,JC066,40.744318,-74.034337,40.727224,-74.033759,member,6.50,1.90
4,B49012F99F97FEE4,classic_bike,2024-04-13 08:50:30,2024-04-13 08:55:10,Lafayette Park,JC078,Liberty Light Rail,JC052,40.713464,-74.062859,40.711242,-74.055701,member,4.67,0.65


In [15]:
dfcitibike24.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159365 entries, 0 to 80687
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             159365 non-null  object 
 1   rideable_type       159365 non-null  object 
 2   started_at          159365 non-null  object 
 3   ended_at            159365 non-null  object 
 4   start_station_name  159365 non-null  object 
 5   start_station_id    159365 non-null  object 
 6   end_station_name    159365 non-null  object 
 7   end_station_id      159365 non-null  object 
 8   start_lat           159365 non-null  float64
 9   start_lng           159365 non-null  float64
 10  end_lat             159365 non-null  float64
 11  end_lng             159365 non-null  float64
 12  user_type           159365 non-null  object 
 13  trip_duration       159365 non-null  object 
 14  distance_in_miles   159365 non-null  object 
dtypes: float64(4), object(11)
memory usage: 

In [16]:
dfcitibike24.to_csv('datalast/citibike24.csv')